In [ ]:
!pip install pandas
!pip install extract-msg
!pip install openpyxl
!pip install pywin32
!pip install beautifulsoup4
!pip install XlsxWriter

In [ ]:
pip install tabulate

## DESCRIPCION : 

El siguiente codigo se encarga de buscar en el correo del estudiante de practicas todos los correos que tengan como asunto las palabras , Veeam Backup y Veeam y Veam y aparte pida el mes ya que los informes son por meses entonces el asunto siempre va a tener alguna de las palabras que se mencionan anteiormente y organizarlos en diferentes hojas y columnas segun corresponda.

# MI PRUEBA DESDE 0

In [ ]:
import win32com.client
import email
from email.header import decode_header
import os
import email.utils
from datetime import datetime
import pytz
import unicodedata

# === Función para normalizar texto con tildes ===
def normalizar(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def leer_eml(archivo):
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            mensaje_eml = email.message_from_file(f)

            # — Decodificar asunto —
            asunto, encoding = decode_header(mensaje_eml["Subject"])[0]
            if isinstance(asunto, bytes):
                asunto = asunto.decode(encoding or 'utf-8')
            print(f"   ✉️ Asunto del .eml: {asunto}")

            # — Buscar y mostrar solo la primera fecha con zona -0500 (-05) —
            received_headers = mensaje_eml.get_all("Received")
            fecha_objetivo = None

            if received_headers:
                for line in reversed(received_headers):
                    if ";" in line:
                        raw_fecha = line.split(";")[-1].strip()
                        if "-0500" in raw_fecha and "(-05)" in raw_fecha:
                            parsed = email.utils.parsedate_tz(raw_fecha)
                            if parsed:
                                timestamp = email.utils.mktime_tz(parsed)
                                dt = datetime.fromtimestamp(timestamp, pytz.timezone("America/Bogota"))
                                fecha_objetivo = dt.strftime('%a %Y-%m-%d %H:%M')
                                break

            if fecha_objetivo:
                print(f"   🕒 Hora real de recibido en COT: {fecha_objetivo}")
            else:
                print("   ⚠️ No se encontró una fecha con zona -0500 (-05).")

            # — Extraer cuerpo —
            cuerpo = ""
            if mensaje_eml.is_multipart():
                for parte in mensaje_eml.walk():
                    ctype = parte.get_content_type()
                    cdispo = str(parte.get("Content-Disposition"))
                    if ctype == "text/plain" and "attachment" not in cdispo:
                        charset = parte.get_content_charset() or "utf-8"
                        cuerpo = parte.get_payload(decode=True).decode(charset, errors="ignore")
                        break
                    elif ctype == "text/html" and "attachment" not in cdispo:
                        charset = parte.get_content_charset() or "utf-8"
                        cuerpo = parte.get_payload(decode=True).decode(charset, errors="ignore")
                        break
            else:
                charset = mensaje_eml.get_content_charset() or "utf-8"
                cuerpo = mensaje_eml.get_payload(decode=True).decode(charset, errors="ignore")

            if cuerpo:
                print("   🧾 Cuerpo del correo (contenido completo):")
                print(cuerpo)
            else:
                print("   ⚠️ No se encontró cuerpo de texto legible.")
    except Exception as e:
        print(f"⚠️ Error al leer el archivo .eml con Python: {e}")

# === Solicitar datos al usuario ===
mes = input("Ingresa el mes (Ej: Abril): ").capitalize()
anio = input("Ingresa el año (Ej: 2025): ")
dia_inicio = int(input("Día de inicio (Ej: 1): "))
dia_fin = int(input("Día de fin (Ej: 31): "))

# === Preparar valores para búsqueda ===
meses_esp = {
    "Enero": "January", "Febrero": "February", "Marzo": "March", "Abril": "April",
    "Mayo": "May", "Junio": "June", "Julio": "July", "Agosto": "August",
    "Septiembre": "September", "Octubre": "October", "Noviembre": "November", "Diciembre": "Diciembre"
}
mes_num = list(meses_esp.keys()).index(mes) + 1

# Zona horaria de Colombia (COT)
colombia_tz = pytz.timezone("America/Bogota")

# === Conectar a Outlook ===
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
buzon = outlook.Folders.Item("epseguridadti@olimpica.com.co")  # Ajusta si es otro buzón
carpeta = buzon.Folders["Veam BK"]

# === Obtener correos ===
mensajes = carpeta.Items
mensajes.Sort("[ReceivedTime]", True)

print("\n📨 Correos encontrados:")
for mensaje in mensajes:
    try:
        asunto = normalizar(mensaje.Subject)
        recibido = mensaje.ReceivedTime

        if "Veeam" in asunto and mes in asunto and anio in asunto and dia_inicio <= recibido.day <= dia_fin and recibido.month == mes_num:
            # Convertir la fecha de recepción a zona horaria de Colombia
            recibido_colombia = recibido.astimezone(colombia_tz)

            print(f"\n🕒 Recibido: {recibido_colombia.strftime('%d/%m/%Y %H:%M')}")
            print(f"Asunto: {mensaje.Subject}")

            # Revisar si hay adjuntos
            if mensaje.Attachments.Count > 0:
                print("📎 Adjuntos encontrados:")
                for i in range(1, mensaje.Attachments.Count + 1):
                    adjunto = mensaje.Attachments.Item(i)
                    nombre = adjunto.FileName
                    print(f" - {nombre}")

                    if nombre.lower().endswith(".eml"):
                        print("   🔍 Analizando contenido del .eml...")

                        # Crear carpeta para guardar archivos .eml
                        carpeta_destino = os.path.join(os.getcwd(), "Archivos_eml")
                        if not os.path.exists(carpeta_destino):
                            os.makedirs(carpeta_destino)
                            print(f"📂 Carpeta '{carpeta_destino}' creada para guardar los archivos .eml")

                        # Guardar archivo .eml en la carpeta
                        ruta_temp = os.path.join(carpeta_destino, nombre)
                        adjunto.SaveAsFile(ruta_temp)

                        # Leer el archivo .eml
                        leer_eml(ruta_temp)

                        # Eliminar archivo .eml después de procesarlo
                        os.remove(ruta_temp)
                        print(f"🗑️ Archivo {nombre} eliminado después de procesarlo.")
    except Exception as e:
        print(f"⚠️ Error leyendo un mensaje: {e}")


# intento 2.0

In [1]:
from datetime import datetime
import pytz
import os
import unicodedata
import win32com.client
import email
from email.header import decode_header
import email.utils

# === Funciones auxiliares ===
def normalizar(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def leer_eml(archivo, archivo_salida):
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            mensaje_eml = email.message_from_file(f)

            asunto, encoding = decode_header(mensaje_eml["Subject"])[0]
            if isinstance(asunto, bytes):
                asunto = asunto.decode(encoding or 'utf-8')
            print(f"   ✉️ Asunto del .eml: {asunto}")
            archivo_salida.write(f"   ✉️ Asunto del .eml: {asunto}\n")

            received_headers = mensaje_eml.get_all("Received")
            fecha_objetivo = None

            if received_headers:
                for line in reversed(received_headers):
                    if ";" in line:
                        raw_fecha = line.split(";")[-1].strip()
                        if "-0500" in raw_fecha and "(-05)" in raw_fecha:
                            parsed = email.utils.parsedate_tz(raw_fecha)
                            if parsed:
                                timestamp = email.utils.mktime_tz(parsed)
                                dt = datetime.fromtimestamp(timestamp, pytz.timezone("America/Bogota"))
                                fecha_objetivo = dt.strftime('%a %Y-%m-%d %H:%M')
                                break

            if fecha_objetivo:
                print(f"   🕒 Hora real de recibido en COT: {fecha_objetivo}")
                archivo_salida.write(f"   🕒 Hora real de recibido en COT: {fecha_objetivo}\n")
            else:
                print("   ⚠️ No se encontró una fecha con zona -0500 (-05).")
                archivo_salida.write("   ⚠️ No se encontró una fecha con zona -0500 (-05).\n")

            cuerpo = ""
            if mensaje_eml.is_multipart():
                for parte in mensaje_eml.walk():
                    ctype = parte.get_content_type()
                    cdispo = str(parte.get("Content-Disposition"))
                    if ctype == "text/plain" and "attachment" not in cdispo:
                        charset = parte.get_content_charset() or "utf-8"
                        cuerpo = parte.get_payload(decode=True).decode(charset, errors="ignore")
                        break
            else:
                charset = mensaje_eml.get_content_charset() or "utf-8"
                cuerpo = mensaje_eml.get_payload(decode=True).decode(charset, errors="ignore")

            if cuerpo:
                print("   🧾 Cuerpo del correo:")
                print(cuerpo)
                archivo_salida.write("   🧾 Cuerpo del correo:\n")
                archivo_salida.write(cuerpo + "\n")
            else:
                print("   ⚠️ No se encontró cuerpo legible.")
                archivo_salida.write("   ⚠️ No se encontró cuerpo legible.\n")

    except Exception as e:
        print(f"⚠️ Error al leer .eml: {e}")
        archivo_salida.write(f"⚠️ Error al leer .eml: {e}\n")

# === Inputs ===
mes = input("Ingresa el mes (Ej: Abril): ").capitalize()
anio = input("Ingresa el año (Ej: 2025): ")
dia_inicio = int(input("Día de inicio (Ej: 1): "))
dia_fin = int(input("Día de fin (Ej: 31): "))

meses_esp = {
    "Enero": "January", "Febrero": "February", "Marzo": "March", "Abril": "April",
    "Mayo": "May", "Junio": "June", "Julio": "July", "Agosto": "August",
    "Septiembre": "September", "Octubre": "October", "Noviembre": "November", "Diciembre": "December"
}
mes_num = list(meses_esp.keys()).index(mes) + 1
colombia_tz = pytz.timezone("America/Bogota")

# === Outlook ===
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
buzon = outlook.Folders.Item("epseguridadti@olimpica.com.co")
carpeta = buzon.Folders["Veam BK"]
mensajes = carpeta.Items
mensajes.Sort("[ReceivedTime]", True)

# === A partir de aquí, también se guarda en salida.txt ===
print("\n📨 Correos encontrados:")

with open("salida.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write("📨 Correos encontrados:\n")

    for mensaje in mensajes:
        try:
            asunto = normalizar(mensaje.Subject)
            recibido = mensaje.ReceivedTime

            if "Veeam" in asunto and mes in asunto and anio in asunto and dia_inicio <= recibido.day <= dia_fin and recibido.month == mes_num:
                recibido_colombia = recibido.astimezone(colombia_tz)

                print(f"\n🕒 Recibido: {recibido_colombia.strftime('%d/%m/%Y %H:%M')}")
                print(f"Asunto: {mensaje.Subject}")
                archivo_salida.write(f"\n🕒 Recibido: {recibido_colombia.strftime('%d/%m/%Y %H:%M')}\n")
                archivo_salida.write(f"Asunto: {mensaje.Subject}\n")

                if mensaje.Attachments.Count > 0:
                    print("📎 Adjuntos encontrados:")
                    archivo_salida.write("📎 Adjuntos encontrados:\n")
                    for i in range(1, mensaje.Attachments.Count + 1):
                        adjunto = mensaje.Attachments.Item(i)
                        nombre = adjunto.FileName
                        print(f" - {nombre}")
                        archivo_salida.write(f" - {nombre}\n")

                        if nombre.lower().endswith(".eml"):
                            print("   🔍 Analizando contenido del .eml...")
                            archivo_salida.write("   🔍 Analizando contenido del .eml...\n")

                            carpeta_destino = os.path.join(os.getcwd(), "Archivos_eml")
                            if not os.path.exists(carpeta_destino):
                                os.makedirs(carpeta_destino)

                            ruta_temp = os.path.join(carpeta_destino, nombre)
                            adjunto.SaveAsFile(ruta_temp)

                            leer_eml(ruta_temp, archivo_salida)

                            os.remove(ruta_temp)
                            print(f"🗑️ Archivo {nombre} eliminado.")
                            archivo_salida.write(f"🗑️ Archivo {nombre} eliminado.\n")
        except Exception as e:
            print(f"⚠️ Error leyendo mensaje: {e}")
            archivo_salida.write(f"⚠️ Error leyendo mensaje: {e}\n")


Ingresa el mes (Ej: Abril): mayo
Ingresa el año (Ej: 2025): 2025
Día de inicio (Ej: 1): 1
Día de fin (Ej: 31): 31

📨 Correos encontrados:

🕒 Recibido: 23/05/2025 03:52
Asunto: FW: Informe Final Veeam Backup - Olímpica S.A. 21 de Mayo 2025
📎 Adjuntos encontrados:
 - Informe Final Veeam Backup - Olímpica S.A. 21 de Mayo 2025.eml
   🔍 Analizando contenido del .eml...
   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 21 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 22:20
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.


Replication
Observaciones
Porcentaje completado
71%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
22
Jobs No Replicados
9
Hora de inicio de la ventana
22/05/2025 - 12:00 AM
Hora de finalización de la ventana
                 22/05/2025 - 9:03 PM
Tiempo de backup
                                        21h 03m


NOTA: Con el 

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 21 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 16:25
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
21/5/2025 - 7:00 PM
Hora de finalización de la ventana
21/05/2025 - 10:41 PM
Tiempo de Backup
3h 41m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
66.4 TB
TB no respaldadas
0 TB
Hora de inicio
21/05/2025 - 10:46 PM
Hora de finalización
22/05/2025 - 3:52 PM
Tiempo de Backup
17h 05m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
56%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Replica

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 21 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 10:51
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
21/5/2025 - 7:00 PM
Hora de finalización de la ventana
21/05/2025 - 10:41 PM
Tiempo de Backup
3h 41m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
96%
ESPERANDO POR CINTA
Porcentaje faltante
4%
TB respaldadas
66.2 TB
TB no respaldadas
2.2 TB
Hora de inicio
21/05/2025 - 10:46 PM
Hora de finalización

Tiempo de Backup
11h 50m
Estado
EN EJECUCIÓN
Replication
Observaciones
Porcentaje completado
56%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Replicados
17
J

 - Informe Final Veeam Backup - Olímpica S.A. 20 de Mayo 2025.eml
   🔍 Analizando contenido del .eml...
   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 20 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 06:16
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.


Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
21/05/2025 - 12:00 AM
Hora de finalización de la ventana
     21/05/2025 - 11:11 PM
Tiempo de backup
                              23h 11m



Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional


🕒 Recibido: 22/05/2025 03:13
Asunto: FW: Informe Veeam Backup - Olímpica S.A. 20 de Mayo 2025
📎 Adjuntos encontrados:
 - Informe Veeam Backup - Olímpica S.A. 20 de Mayo 2025.eml
   🔍 Analizando contenido del .eml...
   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 20 de Mayo 2025
   🕒 Hora real de recibido en COT: Wed 2025-05-21 16:59
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
20/5/2025 - 7:00 PM
Hora de finalización de la ventana
20/05/2025 - 10:27 PM
Tiempo de Backup
3h 27m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
66.3 TB
TB no respaldadas
0 TB
Hora de 

...

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 3 de Mayo 2025
   🕒 Hora real de recibido en COT: Sun 2025-05-04 19:23
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
3/5/2025 - 7:00 PM
Hora de finalización de la ventana
         4/5/2025 - 1:11 AM
Tiempo de Backup
6h 11m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.7 TB
TB no respaldadas
0 TB
Hora de inicio
      4/5/2025 - 1:11 AM
Hora de finalización
4/5/2025 - 1:27 PM
Tiempo de Backup
          12h 15m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
100%
FINALIZADO
Total de Jobs Programados


   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 2 de Mayo 2025
   🕒 Hora real de recibido en COT: Sun 2025-05-04 00:44
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.6 TB
TB no respaldadas
0 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización
3/5/2025 - 12:01 PM
Tiempo de Backup
          12h 12m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Prog

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 2 de Mayo 2025
   🕒 Hora real de recibido en COT: Sat 2025-05-03 11:01
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente

El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_MENSUAL
  Observaciones
Porcentaje completado
98%
EN EJECUCIÓN
Porcentaje faltante
2%
TB respaldadas
64.5 TB
TB no respaldadas
1.1 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización

Tiempo de Backup
          10h 59m
Estado
RUNNING
Replication
Observaciones
Porcentaje completado
52%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Repli

   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 1 de Mayo 2025
   🕒 Hora real de recibido en COT: Fri 2025-05-02 19:06
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente

El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.


Replication
Observaciones
Porcentaje completado
98%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
2/5/2025 - 12:00 AM
Hora de finalización de la ventana
2/5/2025 - 6:15 PM
Tiempo de backup
                       18h 15m


Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:0feee93a-f2fc-4ef5-9875-a327e359238d]

Centro de Ayuda a Clien

   ✉️ Asunto del .eml: Informe Final Mensual Veeam Backup - Olímpica S.A. 30 de Abril 2025
   🕒 Hora real de recibido en COT: Fri 2025-05-02 05:23
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente.


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
1/5/2025 - 01:45 AM
Hora de finalización de la ventana
   2/5/2025 - 2:06 AM
Tiempo de backup
                       24h 21m



Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:48514d4c-5a26-4534-bdcc-2914a395d92b]

Centro d

## ORGANIZAR INFORMACION DE CUERPO DE CORREOS SALIDA.TXT

In [15]:
import re
import pandas as pd
from datetime import datetime
import platform

def obtener_formato_fecha():
    return "%#d-%b" if platform.system() == "Windows" else "%-d-%b"

formato_fecha = obtener_formato_fecha()

def extraer_fecha_y_hora_real(correo):
    match = re.search(r"Hora real de recibido en COT: \w{3} (\d{4}-\d{2}-\d{2}) (\d{2}:\d{2})", correo)
    if match:
        dt = datetime.strptime(match.group(1), "%Y-%m-%d")
        fecha = dt.strftime(formato_fecha).capitalize()
        return fecha, match.group(2)
    return "N/A", "N/A"

def limpiar(valor):
    return valor.strip().replace("\xa0", " ") if valor else "N/A"

def formatear_fecha_estilo(fecha_str):
    try:
        dt = datetime.strptime(fecha_str, "%d/%m/%Y")
        return dt.strftime(formato_fecha).capitalize()
    except:
        return "N/A"

def formatear_fecha_normal(fecha_str):
    try:
        dt = datetime.strptime(fecha_str, "%d/%m/%Y")
        return dt.strftime("%d/%m/%Y")
    except:
        return "N/A"

# Leer archivo salida.txt
with open("salida.txt", "r", encoding="utf-8") as f:
    contenido = f.read()

correos = re.split(r"🗑️ Archivo .*? eliminado después de procesarlo\.", contenido)

# Inicializar listas
backup_disco, backup_cinta, tape_mensual, replication = [], [], [], []

for correo in correos:
    fecha, hora = extraer_fecha_y_hora_real(correo)

    for match in re.finditer(
        r"Backup a disco.*?Porcentaje completado\n(\d+)%\n(.*?)\nPorcentaje faltante\n(\d+)%\n"
        r"Cantidad MV respaldadas.*?\n(\d+)\nCantidad MV no respaldadas.*?\n(\d+)\n"
        r"Hora de inicio de la ventana\n([^\n]+)\nHora de finalización de la ventana\n([^\n]+)\n"
        r"Tiempo de Backup\n([^\n]+)", correo, re.DOTALL):

        fecha_raw = limpiar(match.group(6).split("-")[0])
        fecha_real = formatear_fecha_estilo(fecha_raw)
        fecha_inicio_normal = formatear_fecha_normal(fecha_raw)

        backup_disco.append({
            "Fecha Real": fecha_real,
            "Fecha": fecha,
            "Hora": hora,
            "Porcentaje completado": f"{match.group(1)}%",
            "Porcentaje faltante": f"{match.group(3)}%",
            "Cantidad MV respaldadas de Jobs ejecutados": match.group(4),
            "Cantidad MV no respaldadas de los Jobs ejecutados": match.group(5),
            "Fecha de inicio de la ventana": fecha_inicio_normal,
            "Hora de inicio de la ventana": limpiar(match.group(6).split("-")[-1]),
            "Fecha de finalización de la ventana": limpiar(match.group(7).split("-")[0]),
            "Hora de finalización de la ventana": limpiar(match.group(7).split("-")[-1]),
            "Tiempo de Backup": limpiar(match.group(8)),
            "Estado": limpiar(match.group(2)),
            "Actividad": "Backup a disco - Veeam"
        })

    for match in re.finditer(
        r"Backup a cinta.*?Porcentaje completado\n(\d+)%\n(.*?)\nPorcentaje faltante\n(\d+)%\n"
        r"TB respaldadas\n([^\n]+) TB\nTB no respaldadas\n([^\n]+) TB\n"
        r"Hora de inicio\n([^\n]+)\nHora de finalización\n([^\n]*)\n"
        r"Tiempo de Backup\n([^\n]+)\nEstado\n([^\n]+)", correo, re.DOTALL):

        fecha_raw = limpiar(match.group(6).split("-")[0])
        fecha_real = formatear_fecha_estilo(fecha_raw)
        fecha_inicio_normal = formatear_fecha_normal(fecha_raw)

        backup_cinta.append({
            "Fecha Real": fecha_real,
            "Fecha": fecha,
            "Hora": hora,
            "Porcentaje completado": f"{match.group(1)}%",
            "Porcentaje faltante": f"{match.group(3)}%",
            "TB respaldadas": f"{match.group(4).replace(',', '.')} TB",
            "TB no respaldadas": f"{match.group(5).replace(',', '.')} TB",
            "Fecha de inicio": fecha_inicio_normal,
            "Hora de inicio": limpiar(match.group(6).split("-")[-1]),
            "Fecha de finalización": limpiar(match.group(7).split("-")[0]) if match.group(7) else "N/A",
            "Hora de finalización": limpiar(match.group(7).split("-")[-1]) if match.group(7) else "N/A",
            "Tiempo de Backup": limpiar(match.group(8)),
            "Estado": limpiar(match.group(9)),
            "Actividad": "Backup a cinta - TAPE_DIARIO_NEW"
        })

    # Tape Mensual
    for match in re.finditer(
        r"TAPE_MENSUAL_NEW.*?Porcentaje completado\n(\d+)%\n(.*?)\nPorcentaje faltante\n(\d+)%\n"
        r"TB respaldadas\n([^\n]+) TB\nTB no respaldadas\n([^\n]+) TB\n"
        r"Fecha de inicio de la ventana\n([^\n]+)\nHora de inicio.*?\n([^\n]+)\n"
        r"Fecha de finalización.*?\n([^\n]+)\nHora de finalización.*?\n([^\n]+)\n"
        r"Tiempo de Backup\n([^\n]+)", correo, re.DOTALL):

        fecha_raw = limpiar(match.group(6))
        fecha_real = formatear_fecha_estilo(fecha_raw)
        fecha_inicio_normal = formatear_fecha_normal(fecha_raw)

        tape_mensual.append({
            "Fecha Real": fecha_real,
            "Fecha": fecha,
            "Hora": hora,
            "Porcentaje completado": f"{match.group(1)}%",
            "Porcentaje faltante": f"{match.group(3)}%",
            "TB respaldadas": f"{match.group(4).replace(',', '.')} TB",
            "TB no respaldadas": f"{match.group(5).replace(',', '.')} TB",
            "Fecha de inicio de la ventana": fecha_inicio_normal,
            "Hora de inicio de la ventana": limpiar(match.group(7)),
            "Fecha de finalización de la ventana": limpiar(match.group(8)),
            "Hora de finalización de la ventana": limpiar(match.group(9)),
            "Tiempo de Backup": limpiar(match.group(10)),
            "Estado": limpiar(match.group(2)),
            "Actividad": "TAPE_MENSUAL_NEW"
        })

    # Replication
    for match in re.finditer(
        r"Replication.*?Porcentaje completado\n(\d+)%\n(.*?)\nTotal de Jobs.*?\n(\d+)\n"
        r"Jobs Replicados\n(\d+)\nJobs No Replicados\n(\d+)\n"
        r"Hora de inicio de la ventana\n([^\n]+)\nHora de finalización de la ventana\n([^\n]*)\n"
        r"Tiempo de backup\n\s+([^\n]+)", correo, re.DOTALL):

        fecha_raw = limpiar(match.group(6).split("-")[0])
        fecha_real = formatear_fecha_estilo(fecha_raw)
        fecha_inicio_normal = formatear_fecha_normal(fecha_raw)

        replication.append({
            "Fecha Real": fecha_real,
            "Fecha": fecha,
            "Hora": hora,
            "Porcentaje completado": f"{match.group(1)}%",
            "Total de Jobs Programados": match.group(3),
            "Jobs Replicados": match.group(4),
            "Jobs No Replicados": match.group(5),
            "Fecha de inicio de la ventana": fecha_inicio_normal,
            "Hora de inicio de la ventana": limpiar(match.group(6).split("-")[-1]),
            "Fecha de finalización de la ventana": limpiar(match.group(7).split("-")[0]) if match.group(7) else "N/A",
            "Hora de finalización de la ventana": limpiar(match.group(7).split("-")[-1]) if match.group(7) else "N/A",
            "Estado": limpiar(match.group(2)),
            "Actividad": "Replication"
        })

# DataFrames y exportar
df_disco = pd.DataFrame(backup_disco)
df_cinta = pd.DataFrame(backup_cinta)
df_mensual = pd.DataFrame(tape_mensual)
df_repli = pd.DataFrame(replication)

# Ordenar por fecha y hora
for df in [df_disco, df_cinta, df_mensual, df_repli]:
    df.fillna("N/A", inplace=True)
    if not df.empty:
        df["FechaHora"] = pd.to_datetime(df["Fecha"] + " " + df["Hora"], dayfirst=True, errors="coerce")
        df.sort_values("FechaHora", inplace=True)
        df.drop(columns=["FechaHora"], inplace=True)
        df.drop_duplicates(subset=["Fecha Real", "Actividad", "Estado"], keep="first", inplace=True)

# Exportar
with pd.ExcelWriter("veeam_backup_organizado.xlsx", engine="xlsxwriter") as writer:
    df_disco.to_excel(writer, sheet_name="Backup a Disco", index=False)
    df_cinta.to_excel(writer, sheet_name="Backup a cinta", index=False)
    df_mensual.to_excel(writer, sheet_name="Tape mensual", index=False)
    df_repli.to_excel(writer, sheet_name="Replication", index=False)


# PRUEBA DE GEMINI - PROCESAMIENTO TXT

In [41]:
import re
import pandas as pd
from datetime import datetime
import pytz
import os
import unicodedata
import win32com.client
import email
from email.header import decode_header
import email.utils

# --- Funciones auxiliares ---
def normalizar(texto):
    """Normaliza un texto quitando tildes y caracteres especiales."""
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def round_to_nearest_minute(dt_obj):
    """
    Redondea un objeto datetime al minuto más cercano.
    Si los segundos son 30 o más, redondea al siguiente minuto.
    Si los segundos son menos de 30, trunca los segundos al minuto actual.
    """
    if dt_obj.second >= 30:
        # Sube al siguiente minuto y elimina segundos y microsegundos
        dt_obj = dt_obj.replace(second=0, microsecond=0) + pd.Timedelta(minutes=1)
    else:
        # Trunca los segundos y microsegundos
        dt_obj = dt_obj.replace(second=0, microsecond=0)
    return dt_obj

def leer_eml(archivo, archivo_salida, colombia_tz):
    """
    Lee un archivo .eml, extrae información relevante como asunto,
    hora de envío y cuerpo, y la escribe en el archivo de salida.
    """
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            mensaje_eml = email.message_from_file(f)

            # --- Asunto del EML adjunto ---
            asunto, encoding = decode_header(mensaje_eml["Subject"])[0]
            if isinstance(asunto, bytes):
                asunto = asunto.decode(encoding or 'utf-8', errors='ignore')
            print(f"    ✉️ Asunto del .eml: {asunto}")
            archivo_salida.write(f"    ✉️ Asunto del .eml: {asunto}\n")

            # --- ANALISIS DEL ENCABEZADO "Date" (Hora de Envío Original del EML) ---
            fecha_eml_obj_date = None
            if "Date" in mensaje_eml:
                try:
                    parsed_date_from_header = email.utils.parsedate_to_datetime(mensaje_eml["Date"])

                    # Siempre convertir a la zona de Bogotá
                    if parsed_date_from_header.tzinfo is None:
                        # Si es naive, lo asumimos como UTC y lo convertimos a la zona horaria de Colombia
                        fecha_eml_obj_date = pytz.utc.localize(parsed_date_from_header).astimezone(colombia_tz)
                    else:
                        # Si ya tiene tzinfo, solo lo convertimos a la zona horaria de Colombia
                        fecha_eml_obj_date = parsed_date_from_header.astimezone(colombia_tz)

                except Exception as e:
                    print(f"    ⚠️ Error al parsear 'Date' header de EML adjunto: {e}")
                    archivo_salida.write(f"    ⚠️ Error al parsear 'Date' header de EML adjunto: {e}\n")
            else:
                print("    ⚠️ No se encontró 'Date' header en el EML adjunto.")
                archivo_salida.write("    ⚠️ No se encontró 'Date' header en el EML adjunto.\n")

            # --- ANALISIS Y SELECCIÓN DE HORA DE RECEPCIÓN (Received Headers del EML) ---
            received_headers = mensaje_eml.get_all("Received")
            all_parsed_received_dates_in_cot = []

            if received_headers:
                for header_raw in received_headers:
                    try:
                        date_part = header_raw.split(';')[-1].strip()
                        parsed_received_date = email.utils.parsedate_to_datetime(date_part)

                        if parsed_received_date:
                            if parsed_received_date.tzinfo is None:
                                # Si el received header no tiene TZ, asumimos UTC y convertimos a COT
                                received_date_in_bogota = pytz.utc.localize(parsed_received_date).astimezone(colombia_tz)
                            else:
                                # Si ya tiene tzinfo, lo convertimos a COT
                                received_date_in_bogota = parsed_received_date.astimezone(colombia_tz)

                            all_parsed_received_dates_in_cot.append(received_date_in_bogota)

                    except Exception:
                        pass # Silenciar errores de Received headers individuales
            
            # Determinar la hora final del EML, priorizando el Received más reciente y redondeando
            final_eml_time = None
            if all_parsed_received_dates_in_cot:
                latest_received_date_for_display = max(all_parsed_received_dates_in_cot)
                final_eml_time = round_to_nearest_minute(latest_received_date_for_display)
            elif fecha_eml_obj_date:
                final_eml_time = round_to_nearest_minute(fecha_eml_obj_date)

            if final_eml_time:
                print(f"    🕒 Hora de envío (EML Adjunto en COT): {final_eml_time.strftime('%a %Y-%m-%d %H:%M')}")
                archivo_salida.write(f"    🕒 Hora de envío (EML Adjunto en COT): {final_eml_time.strftime('%a %Y-%m-%d %H:%M')}\n")
            else:
                print("    ⚠️ No se pudo determinar la hora de envío del EML adjunto.")
                archivo_salida.write("    ⚠️ No se pudo determinar la hora de envío del EML adjunto.\n")

            # --- Cuerpo del correo ---
            cuerpo = ""
            if mensaje_eml.is_multipart():
                for parte in mensaje_eml.walk():
                    ctype = parte.get_content_type()
                    cdispo = str(parte.get("Content-Disposition"))
                    if ctype == "text/plain" and "attachment" not in cdispo:
                        charset = parte.get_content_charset() or "utf-8"
                        cuerpo = parte.get_payload(decode=True).decode(charset, errors="ignore")
                        break
            else:
                charset = mensaje_eml.get_content_charset() or "utf-8"
                cuerpo = mensaje_eml.get_payload(decode=True).decode(charset, errors="ignore")

            if cuerpo:
                print("    🧾 Cuerpo del correo:")
                print(cuerpo)
                archivo_salida.write("    🧾 Cuerpo del correo:\n")
                archivo_salida.write(cuerpo + "\n")
            else:
                print("    ⚠️ No se encontró cuerpo legible.")
                archivo_salida.write("    ⚠️ No se encontró cuerpo legible.\n")

    except Exception as e:
        print(f"⚠️ Error al leer .eml: {e}")
        archivo_salida.write(f"⚠️ Error al leer .eml: {e}\n")

# --- Inputs del usuario ---
mes = input("Ingresa el mes (Ej: Abril): ").capitalize()
anio = input("Ingresa el año (Ej: 2025): ")
dia_inicio = int(input("Día de inicio (Ej: 1): "))
dia_fin = int(input("Día de fin (Ej: 31): "))

# --- Configuración de zona horaria y mapeo de meses ---
meses_esp = {
    "Enero": "January", "Febrero": "February", "Marzo": "March", "Abril": "April",
    "Mayo": "May", "Junio": "June", "Julio": "July", "Agosto": "August",
    "Septiembre": "September", "Octubre": "October", "Noviembre": "November", "Diciembre": "December"
}
mes_num = list(meses_esp.keys()).index(mes) + 1
colombia_tz = pytz.timezone("America/Bogota")

# --- Conexión a Outlook ---
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
try:
    buzon = outlook.Folders.Item("epseguridadti@olimpica.com.co")
except Exception:
    print("No se encontró el buzón 'epseguridadti@olimpica.com.co'. Intentando usar el buzón por defecto...")
    buzon = outlook.Session.DefaultStore.GetRootFolder()

# --- Acceso a la carpeta y ordenamiento de mensajes ---
carpeta = buzon.Folders["Veam BK"]
mensajes = carpeta.Items
mensajes.Sort("[ReceivedTime]", True) # Ordena por fecha de recepción, descendente

# --- Procesamiento de correos y escritura en salida.txt ---
print("\n📨 Correos encontrados:")

with open("salida.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write("📨 Correos encontrados:\n")

    for mensaje in mensajes:
        try:
            asunto = normalizar(mensaje.Subject)

            raw_received_time_outlook = mensaje.ReceivedTime

            # Convertir la hora de recepción de Outlook a la zona horaria de Bogotá y redondear
            if raw_received_time_outlook.tzinfo is not None:
                # Si tiene tzinfo, lo hacemos naive primero para evitar doble conversión o errores
                received_as_naive = raw_received_time_outlook.replace(tzinfo=None)
            else:
                received_as_naive = raw_received_time_outlook # Ya era naive

            # Localizamos esta hora naive directamente a la zona horaria de Bogotá.
            recibido_localized = colombia_tz.localize(received_as_naive)

            # Redondear el ReceivedTime del correo principal al minuto más cercano
            recibido = round_to_nearest_minute(recibido_localized)

            # --- Filtrar por fecha y asunto ---
            if "Veeam" in asunto and mes in asunto and anio in asunto and \
               dia_inicio <= recibido.day <= dia_fin and recibido.month == mes_num:

                fecha_hora_cot_str = recibido.strftime('%d/%m/%Y %H:%M')

                print(f"\n--- Procesando correo ---")
                print(f"🕒 Recibido (Outlook COT): {fecha_hora_cot_str}")
                print(f"Asunto: {mensaje.Subject}")
                archivo_salida.write(f"\n--- Procesando correo ---\n")
                archivo_salida.write(f"🕒 Recibido: {fecha_hora_cot_str}\n")
                archivo_salida.write(f"Asunto: {mensaje.Subject}\n")

                if mensaje.Attachments.Count > 0:
                    print("📎 Adjuntos encontrados:")
                    archivo_salida.write("📎 Adjuntos encontrados:\n")
                    for i in range(1, mensaje.Attachments.Count + 1):
                        adjunto = mensaje.Attachments.Item(i)
                        nombre = adjunto.FileName
                        print(f" - {nombre}")
                        archivo_salida.write(f" - {nombre}\n")

                        if nombre.lower().endswith(".eml"):
                            print("    🔍 Analizando contenido del .eml...")
                            archivo_salida.write("    🔍 Analizando contenido del .eml...\n")

                            carpeta_destino = os.path.join(os.getcwd(), "Archivos_eml")
                            if not os.path.exists(carpeta_destino):
                                os.makedirs(carpeta_destino)

                            ruta_temp = os.path.join(carpeta_destino, nombre)
                            adjunto.SaveAsFile(ruta_temp)

                            leer_eml(ruta_temp, archivo_salida, colombia_tz) # Pasamos colombia_tz

                            os.remove(ruta_temp)
                            print(f"🗑️ Archivo {nombre} eliminado.")
                            archivo_salida.write(f"🗑️ Archivo {nombre} eliminado.\n")
                else:
                    # Si no hay adjuntos EML, procesa el cuerpo del correo principal
                    cuerpo_principal = mensaje.Body
                    if cuerpo_principal:
                        print("    🧾 Cuerpo del correo principal:")
                        # Limita la impresión a 500 caracteres para no saturar la consola
                        print(cuerpo_principal[:500] + "..." if len(cuerpo_principal) > 500 else cuerpo_principal)
                        archivo_salida.write("    🧾 Cuerpo del correo principal:\n")
                        archivo_salida.write(cuerpo_principal + "\n")
                    else:
                        print("    ⚠️ No se encontró cuerpo legible en el correo principal.")
                        archivo_salida.write("    ⚠️ No se encontró cuerpo legible en el correo principal.\n")
                    # Aunque no se elimine un archivo, se registra que se procesó
                    archivo_salida.write(f"🗑️ Archivo (sin adjunto .eml) procesado para el asunto: {mensaje.Subject}.\n")
        except Exception as e:
            print(f"⚠️ Error procesando mensaje: {e}")
            archivo_salida.write(f"⚠️ Error procesando mensaje: {e}\n")

print("\nProcesamiento de correos finalizado.")

Ingresa el mes (Ej: Abril): Mayo
Ingresa el año (Ej: 2025): 2025
Día de inicio (Ej: 1): 1
Día de fin (Ej: 31): 31

📨 Correos encontrados:

--- Procesando correo ---
🕒 Recibido (Outlook COT): 28/05/2025 07:58
Asunto: FW: Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025
📎 Adjuntos encontrados:
 - Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025.eml
    🔍 Analizando contenido del .eml...
    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Tue 2025-05-27 22:58
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
26/5/2025 - 7:00 PM
Hora de finalización de la ventana
27/05/2025 - 4:17 AM
Tie

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Tue 2025-05-27 06:01
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
26/5/2025 - 7:00 PM
Hora de finalización de la ventana
27/05/2025 - 4:17 AM
Tiempo de Backup
9h 17m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
69%
EN EJECUCIÓN
Porcentaje faltante
31%
TB respaldadas
36 TB
TB no respaldadas
15.4 TB
Hora de inicio
26/05/2025 - 11:25 PM
Hora de finalización

Tiempo de Backup
6h 16m
Estado
EN EJECUCIÓN
Replication
Observaciones
Porcentaje completado
48%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Replicados
15


🗑️ Archivo Informe FInal Veeam Backup - Olímpica S.A. 25 de Mayo 2025.eml eliminado.
 - Informe Veeam Backup - Olímpica S.A. 25 de Mayo 2025.eml
    🔍 Analizando contenido del .eml...
    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 25 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Mon 2025-05-26 11:03
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
25/5/2025 - 7:00 PM
Hora de finalización de la ventana
25/05/2025 - 10:29 PM
Tiempo de Backup
3h 29m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
98%
EN EJECUCIÓN
Porcentaje faltante
2%
TB respaldadas
65.7 TB
TB no respaldadas
0.8 TB
Hora de inicio
25/05/2025 - 1

    ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 24 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-25 22:32
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
25/05/2025 - 1:00 AM
Hora de finalización de la ventana
25/05/2025 - 20:42 PM
Tiempo de backup
                              19h 42m



Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:ecbee6da-8bfd-4219-a713-15019c2c8750]

C

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 23 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-25 07:24
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
24/05/2025 - 1:00 AM
Hora de finalización de la ventana
         25/05/2025 - 12:54 AM
Tiempo de backup
                              23h 58m


Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:33c63b97-40f8-4dd7-bad9-92dda98f5695]


...

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 3 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-04 19:23
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
3/5/2025 - 7:00 PM
Hora de finalización de la ventana
         4/5/2025 - 1:11 AM
Tiempo de Backup
6h 11m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.7 TB
TB no respaldadas
0 TB
Hora de inicio
      4/5/2025 - 1:11 AM
Hora de finalización
4/5/2025 - 1:27 PM
Tiempo de Backup
          12h 15m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
100%
FINALIZADO
Total de Jobs Pro

    ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 2 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-04 00:45
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.6 TB
TB no respaldadas
0 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización
3/5/2025 - 12:01 PM
Tiempo de Backup
          12h 12m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de 

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 2 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sat 2025-05-03 05:52
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente

El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_MENSUAL
  Observaciones
Porcentaje completado
60%
EN EJECUCIÓN
Porcentaje faltante
40%
TB respaldadas
39.4 TB
TB no respaldadas
26.2 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización

Tiempo de Backup
          5h 50m
Estado
RUNNING
Replication
Observaciones
Porcentaje completado
39%
EN EJECUCIÓN
Total de Jobs Programados
31


    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 1 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Fri 2025-05-02 06:32
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
1/5/2025 - 7:00 PM
Hora de finalización de la ventana
                   2/5/2025 - 2:45 PM
Tiempo de Backup
7h 45m
Backup a cinta - TAPE_OLIMPICA_MENSUAL
  Observaciones
Porcentaje completado
83%
EN EJECUCIÓN
Porcentaje faltante
17%
TB respaldadas
42.6 TB
TB no respaldadas
8.3 TB
Hora de inicio
                   1/5/2025 - 11:51 PM
Hora de finalización

Tiempo de Backup
          6h 23m
Estado
RUNNING
Replication
Observaciones
Porcentaje completado
39%
EN EJECUCIÓN
Total 


Procesamiento de correos finalizado.


In [38]:
import re
import pandas as pd
from datetime import datetime
import platform

# --- Funciones auxiliares ---
def obtener_formato_fecha():
    # Detecta el sistema operativo para ajustar el formato de fecha (eliminar ceros iniciales)
    return "%#d-%b" if platform.system() == "Windows" else "%-d-%b"
formato_fecha = obtener_formato_fecha()

def limpiar(valor):
    if not isinstance(valor, str):
        return "N/A"
    valor = re.sub(r'[\n\r\t]+', ' ', valor)  # Reemplazar saltos de línea/tabulaciones con espacios
    valor = re.sub(r'\s+', ' ', valor).strip() # Eliminar espacios múltiples y trim
    valor = valor.replace("\xa0", " ") # Reemplazar espacio no rompible por espacio normal
    return valor if valor else "N/A"

def normalizar_estado(estado_bruto):
    if not isinstance(estado_bruto, str):
        return "N/A"
    return estado_bruto.strip()

def formatear_fecha_estilo(fecha_str):
    fecha_str_cleaned = limpiar(fecha_str)
    if fecha_str_cleaned == "N/A":
        return "N/A"
    current_year = datetime.now().year
    fmts_con_hora = ["%d/%m/%Y %I:%M %p", "%d/%m/%Y %H:%M", "%Y-%m-%d %H:%M"]
    fmts_con_year = [
        "%d/%m/%Y", "%Y-%m-%d", "%#d/%m/%Y", "%d/%#m/%Y", "%#d/%#m/%Y",
        "%-d/%m/%Y", "%d/%-m/%Y", "%-d/%-m/%Y", "%d/%m/%y", "%d-%m-%y"
    ]
    fmts_sin_year = ["%d/%m", "%m/%d", "%d-%m", "%m-%d", "%d %b", "%d-%b"]

    for fmt in fmts_con_hora:
        try:
            dt = datetime.strptime(fecha_str_cleaned, fmt)
            return dt.strftime(formato_fecha).capitalize()
        except ValueError:
            pass
    for fmt in fmts_con_year:
        try:
            dt = datetime.strptime(fecha_str_cleaned, fmt)
            return dt.strftime(formato_fecha).capitalize()
        except ValueError:
            pass
    for fmt in fmts_sin_year:
        try:
            # Añadir año actual para fechas sin año
            dt = datetime.strptime(f"{fecha_str_cleaned}-{current_year}", f"{fmt}-%Y")
            return dt.strftime(formato_fecha).capitalize()
        except ValueError:
            pass
    return "N/A"

def formatear_fecha_normal(fecha_str):
    fecha_str_cleaned = limpiar(fecha_str)
    if fecha_str_cleaned == "N/A":
        return "N/A"
    current_year = datetime.now().year
    fmts_con_hora = ["%d/%m/%Y %I:%M %p", "%d/%m/%Y %H:%M", "%Y-%m-%d %H:%M"]
    fmts_con_year = [
        "%d/%m/%Y", "%Y-%m-%d", "%#d/%m/%Y", "%d/%#m/%Y", "%#d/%#m/%Y",
        "%-d/%m/%Y", "%d/%-m/%Y", "%-d/%-m/%Y", "%d/%m/%y", "%d-%m-%y"
    ]
    fmts_sin_year = ["%d/%m", "%m/%d", "%d-%m", "%m-%d", "%d %b", "%d-%b"]

    for fmt in fmts_con_hora:
        try:
            dt = datetime.strptime(fecha_str_cleaned, fmt)
            return dt.strftime("%d/%m/%Y")
        except ValueError:
            pass
    for fmt in fmts_con_year:
        try:
            dt = datetime.strptime(fecha_str_cleaned, fmt)
            return dt.strftime("%d/%m/%Y")
        except ValueError:
            pass
    for fmt in fmts_sin_year:
        try:
            dt = datetime.strptime(f"{fecha_str_cleaned}-{current_year}", f"{fmt}-%Y")
            return dt.strftime("%d/%m/%Y")
        except ValueError:
            pass
    return "N/A"

# --- Funciones para extraer datos de cada tipo de backup ---

def extraer_backup_disco(correo_texto, fecha_correo_principal, hora_correo_principal):
    data = []
    pattern = re.compile(
        r"Backup a disco - (?:veeam|Veeam)\s*\n"
        r"Observaciones\s*\n"
        r"Porcentaje completado\s*\n(\d+)%\s*\n"
        r"([^\n]+)\s*\n" # Estado observado (grupo 2)
        r"Porcentaje faltante\s*\n(\d+)%\s*\n"
        r"Cantidad MV respaldadas de Jobs ejecutados\s*\n(\d+)\s*\n"
        r"Cantidad MV no respaldadas de los Jobs ejecutados\s*\n(\d+)\s*\n"
        r"Hora de inicio de la ventana\s*\n([^-\n]+?)\s*-\s*([^\n]+)\s*\n"
        r"Hora de finalización de la ventana\s*\n([^-\n]+?)\s*-\s*([^\n]+)\s*\n"
        r"Tiempo de Backup\s*\n([^\n]+)", re.DOTALL
    )
    for i, match in enumerate(pattern.finditer(correo_texto)):
        print(f"\n--- DEBUG: Backup a Disco Match {i+1} ---")
        print(f"DEBUG: Texto completo del match (Disco):\n'{match.group(0).strip()}'")
        estado_bruto = limpiar(match.group(2)) if match.group(2) else "N/A"
        print(f"DEBUG: Extracted raw status (Disco, group 2): '{estado_bruto}'")
        estado_final = normalizar_estado(estado_bruto)
        print(f"DEBUG: Cleaned and normalized status (Disco): '{estado_final}'")

        fecha_raw_inicio = limpiar(match.group(6)) if match.group(6) else "N/A"
        hora_raw_inicio = limpiar(match.group(7)) if match.group(7) else "N/A"
        fecha_raw_fin = limpiar(match.group(8)) if match.group(8) else "N/A"
        hora_raw_fin = limpiar(match.group(9)) if match.group(9) else "N/A"
        
        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_principal,
            "Hora": hora_correo_principal,
            "Porcentaje completado": f"{limpiar(match.group(1))}%" if match.group(1) else "N/A",
            "Porcentaje faltante": f"{limpiar(match.group(3))}%" if match.group(3) else "N/A",
            "Cantidad MV respaldadas de Jobs ejecutados": limpiar(match.group(4)) if match.group(4) else "N/A",
            "Cantidad MV no respaldadas de los Jobs ejecutados": limpiar(match.group(5)) if match.group(5) else "N/A",
            "Fecha de inicio de la ventana": fecha_actividad_normal,
            "Hora de inicio de la ventana": limpiar(hora_raw_inicio),
            "Fecha de finalización de la ventana": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización de la ventana": limpiar(hora_raw_fin),
            "Tiempo de Backup": limpiar(match.group(10)) if match.group(10) else "N/A",
            "Estado": estado_final,
            "Actividad": "Backup a disco - Veeam"
        })
    return data

def extraer_backup_cinta_generico(correo_texto, fecha_correo_principal, hora_correo_principal):
    data = []
    # MODIFICACIÓN CLAVE AQUÍ:
    # Eliminado el "Estado" al final del bloque para evitar conflictos y confiar solo en el grupo 3.
    # Ajustado el patrón de "Tiempo de Backup" para ser más flexible con los espacios y la ausencia de "Estado".
    pattern = re.compile(
        r"Backup a cinta - ([^\n]+?)\s*\n" # Más específico para el nombre del job
        r"(?:Observaciones\s*\n)?"
        r"Porcentaje completado\s*\n(\d+)%\s*\n"
        r"([^\n]+)\s*\n" # Estado observado (grupo 3) - ESTA ES LA FUENTE PRINCIPAL DEL ESTADO
        r"Porcentaje faltante\s*\n(\d+)%\s*\n"
        r"TB respaldadas\s*\n([^\n]+?)\s*TB\s*\n"
        r"TB no respaldadas\s*\n([^\n]+?)\s*TB\s*\n"
        r"Hora de inicio\s*\n([^-\n]+?)\s*-\s*([^\n]+)\s*\n"
        r"Hora de finalización(?:\s*\n\s*-\s*|\s*\n|\s+)" # Hago la parte de la hora de finalización más flexible
        r"([^-\n]*?)\s*-\s*([^\n]*)\s*\n?" # Puede ser N/A si no hay fecha/hora, hago el último \n opcional
        r"Tiempo de Backup\s*\n\s*([^\n]+)(?:\s*\nEstado\s*\n[^\n]+)?", re.DOTALL # Captura tiempo y hace el "Estado" final opcional
    )
    for i, match in enumerate(pattern.finditer(correo_texto)):
        print(f"\n--- DEBUG: Backup a Cinta Match {i+1} ---")
        print(f"DEBUG: Texto completo del match (Cinta):\n'{match.group(0).strip()}'")
        
        estado_bruto_capturado = limpiar(match.group(3)) if match.group(3) else "N/A"
        print(f"DEBUG: Extracted raw status (Cinta, group 3): '{estado_bruto_capturado}'")
        estado_final = normalizar_estado(estado_bruto_capturado)
        print(f"DEBUG: Cleaned and normalized status (Cinta): '{estado_final}'")

        nombre_job_raw = limpiar(match.group(1)) if match.group(1) else "N/A"
        porcentaje_completado = match.group(2) if match.group(2) else "N/A"
        porcentaje_faltante = match.group(4) if match.group(4) else "N/A"
        tb_respaldadas = limpiar(match.group(5)) if match.group(5) else "N/A"
        tb_no_respaldadas = limpiar(match.group(6)) if match.group(6) else "N/A"
        fecha_raw_inicio = limpiar(match.group(7)) if match.group(7) else "N/A"
        hora_raw_inicio = limpiar(match.group(8)) if match.group(8) else "N/A"
        
        fecha_raw_fin = limpiar(match.group(9)) if match.group(9) else "N/A"
        hora_raw_fin = limpiar(match.group(10)) if match.group(10) else "N/A"
        
        tiempo_backup = limpiar(match.group(11)) if match.group(11) else "N/A" # Este grupo se corrió uno por la eliminación del grupo final de estado

        actividad_determinada = ""
        if "TAPE_MENSUAL_NEW" in nombre_job_raw.upper():
            actividad_determinada = "TAPE_MENSUAL_NEW"
        elif "TAPE_DIARIO_NEW" in nombre_job_raw.upper():
            actividad_determinada = "Backup a cinta - TAPE_DIARIO_NEW"
        elif "TAPE_OLIMPICA_NEW" in nombre_job_raw.upper():
            actividad_determinada = "Backup a cinta - TAPE_OLIMPICA_NEW"
        else:
            actividad_determinada = f"Backup a cinta - {nombre_job_raw}"

        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_principal,
            "Hora": hora_correo_principal,
            "Porcentaje completado": f"{porcentaje_completado}%",
            "Porcentaje faltante": f"{porcentaje_faltante}%",
            "TB respaldadas": f"{tb_respaldadas.replace(',', '.')} TB",
            "TB no respaldadas": f"{tb_no_respaldadas.replace(',', '.')} TB",
            "Fecha de inicio de la ventana": fecha_actividad_normal,
            "Hora de inicio de la ventana": limpiar(hora_raw_inicio),
            "Fecha de finalización de la ventana": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización de la ventana": limpiar(hora_raw_fin),
            "Tiempo de Backup": tiempo_backup,
            "Estado": estado_final,
            "Actividad": actividad_determinada
        })
    return data

def extraer_replication(correo_texto, fecha_correo_principal, hora_correo_principal):
    data = []
    pattern = re.compile(
        r"Replication.*?\s*\nObservaciones\s*\n"
        r"Porcentaje completado\s*\n(\d+)%\s*\n"
        r"([^\n]+)\s*\n" # Estado observado (grupo 2)
        r"Total de Jobs Programados\s*\n(\d+)\s*\n"
        r"Jobs Replicados\s*\n(\d+)\s*\n"
        r"Jobs No Replicados\s*\n(\d+)\s*\n"
        r"Hora de inicio de la ventana\s*\n([^-\n]+?)\s*-\s*([^\n]+)\s*\n"
        r"Hora de finalización de la ventana\s*\n([^-\n]*?)\s*-\s*([^\n]*)\s*\n?" # Hago estos grupos opcionales
        r"Tiempo de backup\s*\n\s*([^\n]+)", re.DOTALL # Flexibilidad para espacios antes del tiempo
    )
    for i, match in enumerate(pattern.finditer(correo_texto)):
        print(f"\n--- DEBUG: Replication Match {i+1} ---")
        print(f"DEBUG: Texto completo del match (Replication):\n'{match.group(0).strip()}'")
        estado_bruto = limpiar(match.group(2)) if match.group(2) else "N/A"
        print(f"DEBUG: Extracted raw status (Replication, group 2): '{estado_bruto}'")
        estado_final = normalizar_estado(estado_bruto)
        print(f"DEBUG: Cleaned and normalized status (Replication): '{estado_final}'")
        
        total_jobs_programados = limpiar(match.group(3)) if match.group(3) else "N/A"
        jobs_replicados = limpiar(match.group(4)) if match.group(4) else "N/A"
        jobs_no_replicados = limpiar(match.group(5)) if match.group(5) else "N/A"
        fecha_raw_inicio = limpiar(match.group(6)) if match.group(6) else "N/A"
        hora_raw_inicio = limpiar(match.group(7)) if match.group(7) else "N/A"
        fecha_raw_fin = limpiar(match.group(8)) if match.group(8) else "N/A"
        hora_raw_fin = limpiar(match.group(9)) if match.group(9) else "N/A"
        tiempo_backup = limpiar(match.group(10)) if match.group(10) else "N/A"

        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_principal,
            "Hora": hora_correo_principal,
            "Porcentaje completado": f"{limpiar(match.group(1))}%" if match.group(1) else "N/A",
            "Total de Jobs Programados": total_jobs_programados,
            "Jobs Replicados": jobs_replicados,
            "Jobs No Replicados": jobs_no_replicados,
            "Fecha de inicio de la ventana": fecha_actividad_normal,
            "Hora de inicio de la ventana": limpiar(hora_raw_inicio),
            "Fecha de finalización de la ventana": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización de la ventana": limpiar(hora_raw_fin),
            "Estado": estado_final,
            "Tiempo de Backup": tiempo_backup,
            "Actividad": "Replication"
        })
    return data

# Leer archivo salida.txt
try:
    with open("salida.txt", "r", encoding="utf-8") as f:
        contenido = f.read()
except FileNotFoundError:
    print("Error: El archivo 'salida.txt' no se encontró. Asegúrate de que la celda 1 se haya ejecutado correctamente.")
    # Define columnas para DataFrames vacíos en caso de error
    column_order_disco_empty = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado", "Porcentaje faltante",
                                 "Cantidad MV respaldadas de Jobs ejecutados", "Cantidad MV no respaldadas de los Jobs ejecutados",
                                 "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                                 "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                                 "Tiempo de Backup", "Estado", "Actividad"]
    column_order_cinta_empty = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado", "Porcentaje faltante",
                                 "TB respaldadas", "TB no respaldadas",
                                 "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                                 "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                                 "Tiempo de Backup", "Estado", "Actividad"]
    column_order_replication_empty = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado",
                                        "Total de Jobs Programados", "Jobs Replicados", "Jobs No Replicados",
                                        "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                                        "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                                        "Estado", "Tiempo de Backup", "Actividad"]

    with pd.ExcelWriter("veeam_backup_organizado_v3.xlsx", engine="xlsxwriter") as writer:
        pd.DataFrame(columns=column_order_disco_empty).to_excel(writer, sheet_name="Backup a Disco", index=False)
        pd.DataFrame(columns=column_order_cinta_empty).to_excel(writer, sheet_name="Backup a cinta", index=False)
        pd.DataFrame(columns=column_order_cinta_empty).to_excel(writer, sheet_name="Tape mensual", index=False)
        pd.DataFrame(columns=column_order_replication_empty).to_excel(writer, sheet_name="Replication", index=False)
    print("Se generó un archivo Excel con hojas vacías.")
    exit()

print("\n--- Contenido leído de salida.txt (primeros 1000 caracteres) ---")
print(contenido[:1000])
print("--------------------------------------------------------------\n")

# Dividir el contenido por cada correo principal
bloques_correo_principal = re.split(r'(?=\n--- Procesando correo ---)', contenido)

all_extracted_data = []

for bloque_correo in bloques_correo_principal:
    if not bloque_correo.strip():
        continue

    # Extraer la fecha y hora del correo principal (Outlook COT)
    match_recibido_principal = re.search(r"🕒 Recibido: (\d{2}/\d{2}/\d{4}) (\d{2}:\d{2})", bloque_correo)
    fecha_correo_principal = "N/A"
    hora_correo_principal = "N/A"

    if match_recibido_principal:
        raw_date_principal = match_recibido_principal.group(1)
        raw_time_principal = match_recibido_principal.group(2)

        try:
            dt_obj_principal = datetime.strptime(raw_date_principal, "%d/%m/%Y")
            fecha_correo_principal = dt_obj_principal.strftime(formato_fecha).capitalize()
            hora_correo_principal = raw_time_principal
        except ValueError:
            pass
    
    print(f"\n--- DEBUG: Procesando el bloque de correo principal con fecha: {fecha_correo_principal}, hora: {hora_correo_principal} ---")
    print(f"DEBUG: Contenido total de este bloque de correo (primeros 500 caracteres):\n'{bloque_correo[:500].strip()}'")

    # Aplicar las funciones de extracción a todo el `bloque_correo`
    # Esto garantiza que se busquen todos los patrones, tanto en el cuerpo principal como en adjuntos/reenvíos.
    all_extracted_data.extend(extraer_backup_disco(bloque_correo, fecha_correo_principal, hora_correo_principal))
    all_extracted_data.extend(extraer_backup_cinta_generico(bloque_correo, fecha_correo_principal, hora_correo_principal))
    all_extracted_data.extend(extraer_replication(bloque_correo, fecha_correo_principal, hora_correo_principal))

# Crear un DataFrame único con todos los datos extraídos
df_all_data = pd.DataFrame(all_extracted_data)

# Si el DataFrame está vacío, salir temprano y generar un Excel vacío
if df_all_data.empty:
    print("No se extrajo ningún dato de los correos. Las hojas de Excel estarán vacías.")
    column_order_disco_empty = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado", "Porcentaje faltante",
                                 "Cantidad MV respaldadas de Jobs ejecutados", "Cantidad MV no respaldadas de los Jobs ejecutados",
                                 "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                                 "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                                 "Tiempo de Backup", "Estado", "Actividad"]
    column_order_cinta_empty = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado", "Porcentaje faltante",
                                 "TB respaldadas", "TB no respaldadas",
                                 "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                                 "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                                 "Tiempo de Backup", "Estado", "Actividad"]
    column_order_replication_empty = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado",
                                        "Total de Jobs Programados", "Jobs Replicados", "Jobs No Replicados",
                                        "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                                        "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                                        "Estado", "Tiempo de Backup", "Actividad"]

    with pd.ExcelWriter("veeam_backup_organizado_v3.xlsx", engine="xlsxwriter") as writer:
        pd.DataFrame(columns=column_order_disco_empty).to_excel(writer, sheet_name="Backup a Disco", index=False)
        pd.DataFrame(columns=column_order_cinta_empty).to_excel(writer, sheet_name="Backup a cinta", index=False)
        pd.DataFrame(columns=column_order_cinta_empty).to_excel(writer, sheet_name="Tape mensual", index=False)
        pd.DataFrame(columns=column_order_replication_empty).to_excel(writer, sheet_name="Replication", index=False)
    print("Se generó un archivo Excel con hojas vacías.")
    exit()

# Agregamos una columna temporal para ordenar por la fecha y hora de la actividad real
df_all_data["FechaHora_Actividad_Real_DT"] = pd.NaT

for index, row in df_all_data.iterrows():
    fecha_str_inicio = row.get("Fecha de inicio de la ventana")
    hora_str_inicio = row.get("Hora de inicio de la ventana")
    
    if fecha_str_inicio != "N/A" and hora_str_inicio != "N/A":
        try:
            dt_obj = datetime.strptime(f"{fecha_str_inicio} {hora_str_inicio}", "%d/%m/%Y %H:%M")
            df_all_data.loc[index, "FechaHora_Actividad_Real_DT"] = dt_obj
        except ValueError:
            pass
            
    # Si la fecha de inicio no se pudo parsear, intentar con la fecha del correo principal
    if pd.isna(df_all_data.loc[index, "FechaHora_Actividad_Real_DT"]):
        fecha_real_estilo = row.get('Fecha Real') # Esta es la fecha del correo principal formateada
        hora_correo_principal = row.get('Hora')
        
        if fecha_real_estilo != "N/A" and hora_correo_principal != "N/A":
            try:
                # Revertir el formato estilo para poder parsear
                current_year = datetime.now().year
                # Intentar parsear el formato "%d-%b" con el año actual
                fecha_estilo_parsed = datetime.strptime(f"{fecha_real_estilo}-{current_year}", "%d-%b-%Y")
                # Crear datetime completo para ordenar
                dt_obj = datetime.strptime(f"{fecha_estilo_parsed.strftime('%d/%m/%Y')} {hora_correo_principal}", "%d/%m/%Y %H:%M")
                df_all_data.loc[index, "FechaHora_Actividad_Real_DT"] = dt_obj
            except ValueError:
                pass

df_final = df_all_data.sort_values(by="FechaHora_Actividad_Real_DT", ascending=True).reset_index(drop=True)

print("\n--- Estados únicos encontrados en los datos extraídos ---")
if not df_final.empty and 'Estado' in df_final.columns:
    print(df_final['Estado'].unique())
else:
    print("No se encontraron estados en los datos extraídos o el DataFrame está vacío.\n")

print("\n--- Primeras 5 filas del DataFrame final (df_final) ---")
print(df_final.head().to_string())
print("-------------------------------------------------------\n")

print("\n--- Últimas 5 filas del DataFrame final (df_final) ---")
print(df_final.tail().to_string())
print("-------------------------------------------------------\n")

print("\n--- Actividades únicas encontradas en los datos extraídos ---")
if not df_final.empty and 'Actividad' in df_final.columns:
    print(df_final['Actividad'].unique())
else:
    print("No se encontraron actividades en los datos extraídos o el DataFrame está vacío.")
print("-----------------------------------------------------------\n")

# Separar los DataFrames por tipo de actividad
df_disco = df_final[df_final["Actividad"] == "Backup a disco - Veeam"].copy()
df_cinta = df_final[
    df_final["Actividad"].str.startswith("Backup a cinta - ", na=False) &
    ~df_final["Actividad"].str.contains("TAPE_MENSUAL_NEW", na=False)
].copy()
df_tape_mensual = df_final[df_final["Actividad"] == "TAPE_MENSUAL_NEW"].copy()
df_replication = df_final[df_final["Actividad"] == "Replication"].copy()

# Función para asegurar que todas las columnas deseadas existen en el DataFrame
def ensure_columns(df, desired_columns):
    for col in desired_columns:
        if col not in df.columns:
            df[col] = "N/A"
    return df[desired_columns] # Reordenar columnas

# Definir el orden deseado de las columnas para cada tipo de reporte
column_order_disco = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado", "Porcentaje faltante",
                      "Cantidad MV respaldadas de Jobs ejecutados", "Cantidad MV no respaldadas de los Jobs ejecutados",
                      "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                      "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                      "Tiempo de Backup", "Estado", "Actividad"]

column_order_cinta = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado", "Porcentaje faltante",
                      "TB respaldadas", "TB no respaldadas",
                      "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                      "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                      "Tiempo de Backup", "Estado", "Actividad"]

column_order_replication = ["Fecha Real", "Fecha", "Hora", "Porcentaje completado",
                            "Total de Jobs Programados", "Jobs Replicados", "Jobs No Replicados",
                            "Fecha de inicio de la ventana", "Hora de inicio de la ventana",
                            "Fecha de finalización de la ventana", "Hora de finalización de la ventana",
                            "Estado", "Tiempo de Backup", "Actividad"]

# Aplicar el orden de las columnas a cada DataFrame
df_disco = ensure_columns(df_disco, column_order_disco)
df_cinta = ensure_columns(df_cinta, column_order_cinta)
df_tape_mensual = ensure_columns(df_tape_mensual, column_order_cinta) # Usa el mismo orden que cinta
df_replication = ensure_columns(df_replication, column_order_replication)

# Guardar los DataFrames en hojas separadas en un archivo Excel
output_excel_file = "veeam_backup_organizado_v3.xlsx"
with pd.ExcelWriter(output_excel_file, engine="xlsxwriter") as writer:
    df_disco.to_excel(writer, sheet_name="Backup a Disco", index=False)
    df_cinta.to_excel(writer, sheet_name="Backup a cinta", index=False)
    df_tape_mensual.to_excel(writer, sheet_name="Tape mensual", index=False)
    df_replication.to_excel(writer, sheet_name="Replication", index=False)

print(f"Datos exportados exitosamente a '{output_excel_file}'")

La hoja 'Backup a cinta' estará vacía porque no se extrajeron datos para esta actividad o no existen en el archivo de entrada.
La hoja 'Tape mensual' estará vacía porque no se extrajeron datos para esta actividad o no existen en el archivo de entrada.
Procesamiento completado y datos exportados a 'veeam_backup_organizado_v3.xlsx'


#copia

In [42]:
import re
import pandas as pd
from datetime import datetime
import platform

# Función para obtener el formato de fecha según el sistema operativo
def obtener_formato_fecha():
    return "%#d-%b" if platform.system() == "Windows" else "%-d-%b"

formato_fecha = obtener_formato_fecha()

def limpiar(valor):
    return re.sub(r'\s+', ' ', valor.strip().replace("\xa0", " ")) if valor else "N/A"

def normalizar_estado(estado_bruto):
    if not isinstance(estado_bruto, str):
        return "N/A"
    estado_upper = estado_bruto.upper()
    if "FINALIZADO" in estado_upper:
        if "ERROR" in estado_upper or "ERRORS" in estado_upper:
            return "FINALIZADO CON ERRORES"
        elif "WARNING" in estado_upper or "ADVERTENCIA" in estado_upper:
            return "FINALIZADO CON ADVERTENCIAS"
        return "FINALIZADO"
    elif "ERROR" in estado_upper:
        return "ERROR"
    elif "EN EJECUCIÓN" in estado_upper or "RUNNING" in estado_upper or "IN PROGRESS" in estado_upper:
        return "EN EJECUCIÓN"
    elif "CANCELADO" in estado_upper:
        return "CANCELADO"
    elif "SUCCESS" in estado_upper:
        return "FINALIZADO"
    return estado_bruto

def formatear_fecha_estilo(fecha_str):
    try:
        fecha_str_cleaned = limpiar(fecha_str)
        for fmt in ["%d/%m/%Y", "%Y-%m-%d", "%#d/%m/%Y", "%d/%#m/%Y", "%#d/%#m/%Y", "%-d/%m/%Y", "%d/%-m/%Y", "%-d/%-m/%Y"]:
            try:
                dt = datetime.strptime(fecha_str_cleaned, fmt)
                return dt.strftime(formato_fecha).capitalize()
            except ValueError:
                pass
        return "N/A"
    except Exception:
        return "N/A"

def formatear_fecha_normal(fecha_str):
    try:
        fecha_str_cleaned = limpiar(fecha_str)
        for fmt in ["%d/%m/%Y", "%Y-%m-%d", "%#d/%m/%Y", "%d/%#m/%Y", "%#d/%#m/%Y", "%-d/%m/%Y", "%d/%-m/%Y", "%-d/%-m/%Y"]:
            try:
                dt = datetime.strptime(fecha_str_cleaned, fmt)
                return dt.strftime("%d/%m/%Y")
            except ValueError:
                pass
        return "N/A"
    except Exception:
        return "N/A"

# --- Funciones para extraer datos de cada tipo de backup ---

def extraer_backup_disco(correo, fecha_correo_eml, hora_correo_eml):
    data = []
    pattern = re.compile(
        r"Backup a disco - (?:veeam|Veeam)\n"
        r"Observaciones\n"
        r"Porcentaje completado\n(\d+)%\n"
        r"(.*?)\n" # Estado observado
        r"Porcentaje faltante\n(\d+)%\n"
        r"Cantidad MV respaldadas de Jobs ejecutados\n(\d+)\n"
        r"Cantidad MV no respaldadas de los Jobs ejecutados\n(\d+)\n"
        r"Hora de inicio de la ventana\n([^\n]+)\n"
        r"Hora de finalización de la ventana\n([^\n]+)\n"
        r"Tiempo de Backup\n([^\n]+)", re.DOTALL
    )
    for match in pattern.finditer(correo):
        estado_observado = limpiar(match.group(2))
        fecha_hora_inicio_raw = limpiar(match.group(6))
        fecha_hora_fin_raw = limpiar(match.group(7))

        try:
            fecha_raw_inicio, hora_raw_inicio = fecha_hora_inicio_raw.split(" - ")
        except ValueError:
            fecha_raw_inicio, hora_raw_inicio = "N/A", "N/A"
        try:
            fecha_raw_fin, hora_raw_fin = fecha_hora_fin_raw.split(" - ")
        except ValueError:
            fecha_raw_fin, hora_raw_fin = "N/A", "N/A"

        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        estado_final = normalizar_estado(estado_observado)

        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_eml,
            "Hora": hora_correo_eml,
            "Porcentaje completado": f"{match.group(1)}%",
            "Porcentaje faltante": f"{match.group(3)}%",
            "Cantidad MV respaldadas de Jobs ejecutados": match.group(4),
            "Cantidad MV no respaldadas de los Jobs ejecutados": match.group(5),
            "Fecha de inicio de la ventana": fecha_actividad_normal,
            "Hora de inicio de la ventana": limpiar(hora_raw_inicio),
            "Fecha de finalización de la ventana": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización de la ventana": limpiar(hora_raw_fin),
            "Tiempo de Backup": limpiar(match.group(8)),
            "Estado": estado_final,
            "Actividad": "Backup a disco - Veeam"
        })
    return data

def extraer_backup_cinta_diario(correo, fecha_correo_eml, hora_correo_eml):
    data = []
    # Busca "Backup a cinta" pero excluye "TAPE_OLIMPICA_MENSUAL" y "TAPE_MENSUAL_NEW" en la misma línea
    pattern = re.compile(
        r"Backup a cinta(?! - TAPE_OLIMPICA_MENSUAL| - TAPE_MENSUAL_NEW).*?\n" # Asegura que no sea Tape Mensual
        r"Porcentaje completado\n(\d+)%\n"
        r"(.*?)\n" # Estado observado (si existe, para ser coherente con Tape Mensual)
        r"Porcentaje faltante\n(\d+)%\n"
        r"TB respaldadas\n([^\n]+) TB\nTB no respaldadas\n([^\n]+) TB\n"
        r"Hora de inicio\n([^\n]+)\nHora de finalización\n([^\n]*)\n"
        r"Tiempo de Backup\n([^\n]+)\n"
        r"(?:Estado\n([^\n]+))?", re.DOTALL # El estado final puede ser opcional
    )
    for match in pattern.finditer(correo):
        # Preferimos el estado observado, si está presente. Si no, usamos el estado final.
        estado_observado = limpiar(match.group(2)) # Primer estado encontrado
        estado_final_del_correo = limpiar(match.group(9)) if match.group(9) else estado_observado # Segundo estado (al final)

        fecha_raw_inicio_hora_raw = limpiar(match.group(6))
        fecha_raw_fin_hora_raw = limpiar(match.group(7))

        try:
            fecha_raw_inicio, hora_raw_inicio = fecha_raw_inicio_hora_raw.split(" - ")
        except ValueError:
            fecha_raw_inicio, hora_raw_inicio = "N/A", "N/A"
        try:
            fecha_raw_fin, hora_raw_fin = fecha_raw_fin_hora_raw.split(" - ") if fecha_raw_fin_hora_raw != "N/A" else ("N/A", "N/A")
        except ValueError:
            fecha_raw_fin, hora_raw_fin = "N/A", "N/A"

        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        estado_para_columna = normalizar_estado(estado_observado if estado_observado != "N/A" else estado_final_del_correo)


        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_eml,
            "Hora": hora_correo_eml,
            "Porcentaje completado": f"{match.group(1)}%",
            "Porcentaje faltante": f"{match.group(3)}%",
            "TB respaldadas": f"{limpiar(match.group(4)).replace(',', '.')} TB",
            "TB no respaldadas": f"{limpiar(match.group(5)).replace(',', '.')} TB",
            "Fecha de inicio": fecha_actividad_normal,
            "Hora de inicio": limpiar(hora_raw_inicio),
            "Fecha de finalización": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización": limpiar(hora_raw_fin),
            "Tiempo de Backup": limpiar(match.group(8)),
            "Estado": estado_para_columna,
            "Actividad": "Backup a cinta - TAPE_DIARIO_NEW" # Esta actividad es fija para esta función
        })
    return data


def extraer_tape_mensual(correo, fecha_correo_eml, hora_correo_eml):
    data = []
    # Busca "TAPE_MENSUAL_NEW" o "TAPE_OLIMPICA_MENSUAL", opcionalmente precedido por "Backup a cinta - "
    pattern = re.compile(
        r"(?:Backup a cinta - )?(TAPE_MENSUAL_NEW|TAPE_OLIMPICA_MENSUAL)\n" # Captura el nombre del job (grupo 1)
        r"(?:Observaciones\n)?" # Observaciones es opcional
        r"Porcentaje completado\n(\d+)%\n"
        r"(.*?)\n" # Estado observado (grupo 3)
        r"Porcentaje faltante\n(\d+)%\n"
        r"TB respaldadas\n([^\n]+) TB\n"
        r"TB no respaldadas\n([^\n]+) TB\n"
        r"Hora de inicio\n([^\n]+)\n" # (grupo 7)
        r"Hora de finalización\n([^\n]+)\n" # (grupo 8)
        r"Tiempo de Backup\n([^\n]+)", re.DOTALL # (grupo 9)
    )
    for match in pattern.finditer(correo):
        nombre_job = limpiar(match.group(1)) # Este es TAPE_MENSUAL_NEW o TAPE_OLIMPICA_MENSUAL
        porcentaje_completado = match.group(2)
        estado_observado = limpiar(match.group(3)) # Estado observado
        porcentaje_faltante = match.group(4)
        tb_respaldadas = limpiar(match.group(5))
        tb_no_respaldadas = limpiar(match.group(6))
        fecha_hora_inicio_raw = limpiar(match.group(7))
        fecha_hora_fin_raw = limpiar(match.group(8))
        tiempo_backup = limpiar(match.group(9))

        try:
            fecha_raw_inicio, hora_raw_inicio = fecha_hora_inicio_raw.split(" - ")
        except ValueError:
            fecha_raw_inicio, hora_raw_inicio = "N/A", "N/A"
        try:
            fecha_raw_fin, hora_raw_fin = fecha_hora_fin_raw.split(" - ")
        except ValueError:
            fecha_raw_fin, hora_raw_fin = "N/A", "N/A"

        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        estado_final = normalizar_estado(estado_observado)

        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_eml,
            "Hora": hora_correo_eml,
            "Porcentaje completado": f"{porcentaje_completado}%",
            "Porcentaje faltante": f"{porcentaje_faltante}%",
            "TB respaldadas": f"{tb_respaldadas.replace(',', '.')} TB",
            "TB no respaldadas": f"{tb_no_respaldadas.replace(',', '.')} TB",
            "Fecha de inicio de la ventana": fecha_actividad_normal,
            "Hora de inicio de la ventana": limpiar(hora_raw_inicio),
            "Fecha de finalización de la ventana": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización de la ventana": limpiar(hora_raw_fin),
            "Tiempo de Backup": tiempo_backup,
            "Estado": estado_final,
            "Actividad": nombre_job # La actividad es TAPE_MENSUAL_NEW o TAPE_OLIMPICA_MENSUAL
        })
    return data

def extraer_replication(correo, fecha_correo_eml, hora_correo_eml):
    data = []
    pattern = re.compile(
        r"Replication.*?\nObservaciones\n"
        r"Porcentaje completado\n(\d+)%\n"
        r"(.*?)\n" # Estado observado
        r"Total de Jobs Programados\n(\d+)\n"
        r"Jobs Replicados\n(\d+)\n"
        r"Jobs No Replicados\n(\d+)\n"
        r"Hora de inicio de la ventana\n([^\n]+)\n"
        r"Hora de finalización de la ventana\n([^\n]*)\n"
        r"Tiempo de backup\n\s*([^\n]+)", re.DOTALL
    )
    for match in pattern.finditer(correo):
        estado_observado = limpiar(match.group(2))
        fecha_hora_inicio_raw = limpiar(match.group(6))
        fecha_hora_fin_raw = limpiar(match.group(7))

        try:
            fecha_raw_inicio, hora_raw_inicio = fecha_hora_inicio_raw.split(" - ")
        except ValueError:
            fecha_raw_inicio, hora_raw_inicio = "N/A", "N/A"
        try:
            fecha_raw_fin, hora_raw_fin = fecha_hora_fin_raw.split(" - ")
        except ValueError:
            fecha_raw_fin, hora_raw_fin = "N/A", "N/A"

        fecha_actividad_estilo = formatear_fecha_estilo(fecha_raw_inicio)
        fecha_actividad_normal = formatear_fecha_normal(fecha_raw_inicio)

        estado_final = normalizar_estado(estado_observado)

        data.append({
            "Fecha Real": fecha_actividad_estilo,
            "Fecha": fecha_correo_eml,
            "Hora": hora_correo_eml,
            "Porcentaje completado": f"{match.group(1)}%",
            "Total de Jobs Programados": match.group(3),
            "Jobs Replicados": match.group(4),
            "Jobs No Replicados": match.group(5),
            "Fecha de inicio de la ventana": fecha_actividad_normal,
            "Hora de inicio de la ventana": limpiar(hora_raw_inicio),
            "Fecha de finalización de la ventana": formatear_fecha_normal(fecha_raw_fin),
            "Hora de finalización de la ventana": limpiar(hora_raw_fin),
            "Estado": estado_final,
            "Tiempo de Backup": limpiar(match.group(8)),
            "Actividad": "Replication"
        })
    return data

# Función para extraer la fecha y hora de recepción del correo .eml (para referencia)
def extraer_fecha_y_hora_del_eml_recibido(correo_texto):
    match_recibido = re.search(r"🕒 Hora real de recibido en COT: \w{3} (\d{4}-\d{2}-\d{2}) (\d{2}:\d{2})", correo_texto)
    if match_recibido:
        raw_date = match_recibido.group(1)
        raw_time = match_recibido.group(2)
        
        dt_obj = datetime.strptime(raw_date, "%Y-%m-%d")
        fecha_formato_estilo = dt_obj.strftime(formato_fecha).capitalize()
        
        return fecha_formato_estilo, raw_time
    return "N/A", "N/A"

# Leer archivo salida.txt
with open("salida.txt", "r", encoding="utf-8") as f:
    contenido = f.read()

bloques_correo = re.split(r"🗑️ Archivo .*? eliminado\.\n", contenido)

# Acumuladores de datos crudos (sin segmentar por tipo aún)
all_extracted_data = []

for bloque_correo in bloques_correo:
    if not bloque_correo.strip():
        continue

    fecha_eml_recibido, hora_eml_recibido = extraer_fecha_y_hora_del_eml_recibido(bloque_correo)

    # Extraer todos los tipos posibles de un bloque
    all_extracted_data.extend(extraer_backup_disco(bloque_correo, fecha_eml_recibido, hora_eml_recibido))
    all_extracted_data.extend(extraer_tape_mensual(bloque_correo, fecha_eml_recibido, hora_eml_recibido))
    all_extracted_data.extend(extraer_backup_cinta_diario(bloque_correo, fecha_eml_recibido, hora_eml_recibido))
    all_extracted_data.extend(extraer_replication(bloque_correo, fecha_eml_recibido, hora_eml_recibido))

# Crear un DataFrame único con todos los datos extraídos
df_all_data = pd.DataFrame(all_extracted_data)

# Si el DataFrame está vacío, salir temprano
if df_all_data.empty:
    print("No se extrajo ningún dato de los correos. Las hojas de Excel estarán vacías.")
    # Exportar archivos vacíos o salir. Para este caso, saldremos sin generar.
    with pd.ExcelWriter("veeam_backup_organizado_v3.xlsx", engine="xlsxwriter") as writer:
        pd.DataFrame().to_excel(writer, sheet_name="Backup a Disco", index=False)
        pd.DataFrame().to_excel(writer, sheet_name="Backup a cinta", index=False)
        pd.DataFrame().to_excel(writer, sheet_name="Tape mensual", index=False)
        pd.DataFrame().to_excel(writer, sheet_name="Replication", index=False)
    print("Se generó un archivo Excel con hojas vacías.")
    exit() # Salir del script

# Filtrar el DataFrame grande en DataFrames específicos para cada hoja
df_disco = df_all_data[df_all_data['Actividad'] == 'Backup a disco - Veeam'].copy()
df_cinta = df_all_data[df_all_data['Actividad'] == 'Backup a cinta - TAPE_DIARIO_NEW'].copy()
df_mensual = df_all_data[
    (df_all_data['Actividad'] == 'TAPE_MENSUAL_NEW') | 
    (df_all_data['Actividad'] == 'TAPE_OLIMPICA_MENSUAL')
].copy()
df_repli = df_all_data[df_all_data['Actividad'] == 'Replication'].copy()

# Definir los nombres de columna para la fecha y hora de inicio de cada actividad
columnas_inicio = {
    "Backup a disco - Veeam": ("Fecha de inicio de la ventana", "Hora de inicio de la ventana"),
    "Backup a cinta - TAPE_DIARIO_NEW": ("Fecha de inicio", "Hora de inicio"),
    "TAPE_MENSUAL_NEW": ("Fecha de inicio de la ventana", "Hora de inicio de la ventana"), 
    "TAPE_OLIMPICA_MENSUAL": ("Fecha de inicio de la ventana", "Hora de inicio de la ventana"),
    "Replication": ("Fecha de inicio de la ventana", "Hora de inicio de la ventana")
}

# Lista de DataFrames y nombres de hoja para el procesamiento final
dataframes_a_procesar_y_exportar = [
    (df_disco, "Backup a Disco"),
    (df_cinta, "Backup a cinta"),
    (df_mensual, "Tape mensual"),
    (df_repli, "Replication")
]

# Proceso de ordenamiento y desduplicación para cada DataFrame
for df, sheet_name in dataframes_a_procesar_y_exportar:
    df.fillna("N/A", inplace=True)
    if not df.empty:
        # La columna 'Actividad' ya está correctamente asignada en el DataFrame
        # Usamos la primera 'Actividad' para determinar las columnas de fecha/hora
        activity_type_for_cols = df['Actividad'].iloc[0]
        fecha_col, hora_col = columnas_inicio.get(activity_type_for_cols, (None, None))

        if fecha_col and hora_col and fecha_col in df.columns and hora_col in df.columns:
            df["FechaHora_Orden"] = pd.to_datetime(
                df[fecha_col] + " " + df[hora_col],
                format="%d/%m/%Y %I:%M %p",
                errors="coerce"
            )
            if df["FechaHora_Orden"].isnull().all():
                 df["FechaHora_Orden"] = pd.to_datetime(
                    df[fecha_col] + " " + df[hora_col],
                    format="%d/%m/%Y %H:%M",
                    errors="coerce"
                )

            df.dropna(subset=["FechaHora_Orden"], inplace=True)
            df.sort_values(by="FechaHora_Orden", ascending=True, inplace=True)

            df_finalizados = df[df["Estado"] == "FINALIZADO"].copy()
            df_otros_estados = df[df["Estado"] != "FINALIZADO"].copy()

            df_finalizados.drop_duplicates(
                subset=[fecha_col, "Actividad"],
                keep="first",
                inplace=True
            )

            df_cleaned = pd.concat([df_finalizados, df_otros_estados]).sort_values(by="FechaHora_Orden", ascending=True)
            
            # Reemplazar el DataFrame original con el limpio y ordenado
            df.drop(df.index, inplace=True) # Vaciar el DF actual
            for col in df_cleaned.columns: # Llenar con datos de df_cleaned
                if col != "FechaHora_Orden":
                    df[col] = df_cleaned[col]
            
            if "FechaHora_Orden" in df.columns:
                df.drop(columns=["FechaHora_Orden"], inplace=True)
        else:
            print(f"Advertencia: No se encontraron columnas de fecha/hora de inicio adecuadas ({fecha_col}, {hora_col}) para la actividad '{activity_type_for_cols}' (hoja: '{sheet_name}'). El DataFrame no se ordenará/desduplicará correctamente.")
    else:
        print(f"La hoja '{sheet_name}' estará vacía porque no se extrajeron datos para esta actividad.")

# Exportar a Excel
with pd.ExcelWriter("veeam_backup_organizado_v3.xlsx", engine="xlsxwriter") as writer:
    df_disco.to_excel(writer, sheet_name="Backup a Disco", index=False)
    df_cinta.to_excel(writer, sheet_name="Backup a cinta", index=False)
    df_mensual.to_excel(writer, sheet_name="Tape mensual", index=False)
    df_repli.to_excel(writer, sheet_name="Replication", index=False)

print("Procesamiento completado y datos exportados a 'veeam_backup_organizado_v3.xlsx'")

La hoja 'Tape mensual' estará vacía porque no se extrajeron datos para esta actividad.
Procesamiento completado y datos exportados a 'veeam_backup_organizado_v3.xlsx'
